In [18]:
import pandas as pd
import os
import os.path as osp

In [ ]:
markdown = '## Leetcode Company-wise Problem Lists\
\n\nCurated lists of Leetcode questions group by companies, updated as of May, 2022.\
\n\nShout out to fishercoder1534 for the awesome [Leetcode](https://github.com/fishercoder1534/Leetcode) repo for solutions.'

leetcode_problems = pd.read_csv('../data/leetcode_problems.csv')  # provides solutions and difficulty levels

companies_dir = '../companies'
for fname in os.listdir(companies_dir):
    fpath = osp.join(companies_dir, fname)
    company_name = fname.split('.')[0]
    df = pd.read_csv(fpath)
    # merge problem_link and problem_name as a markdown link
    problems = []
    for _, row in df.iterrows():
        problem = f"[{row['problem_name']}]({row['problem_link']})"
        problems.append(problem)
    df['problem'] = problems
    # add difficulty and solutions
    df = pd.merge(df, leetcode_problems, left_on='problem_link', right_on='link')
    # drop unnecessary columns
    df = df.drop(['problem_link', 'problem_name', 'name', 'link'], axis = 1)
    # filter numeric rows
    df = df[pd.to_numeric(df.num_occur, errors='coerce').notnull()]
    df['num_occur'] = df['num_occur'].apply(pd.to_numeric, errors='coerce')
    # sort by occurances
    df = df.sort_values(by ='num_occur', ascending=False)
    # filter rows to have >= 3 occurances
    df = df.query('num_occur >= 3')
    # change column name
    df.columns = ['Occurence', 'Problem', 'Difficulty', 'Solution']
    if len(df.index) > 5:
        markdown += f'\n\n## {company_name}\n'
        markdown += df.to_markdown(index=False)

with open('README.md', 'w') as f:
    f.write(markdown)